In [ ]:
!pip install learn2learn

     |████████████████████████████████| 1.4MB 9.3MB/s 
     |████████████████████████████████| 2.9MB 15.8MB/s 
     |████████████████████████████████| 174kB 49.7MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 1.4MB 53.5MB/s 
     |████████████████████████████████| 112kB 54.9MB/s 
     |████████████████████████████████| 3.2MB 47.3MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.5-cp37-cp37m-linux_x86_64.whl size=918077 sha256=7225ba436886e222c88a64e5c9f819d4469e6b30e278390466b8397993c66639
  Stored in directory: /root/.cache/pip/wheels/5a/44/11/9d755af86d74f353dc7e56d5e75668e4ae7f5bf8c7d74f1f7e
  Created wheel for gsutil: filename=gsutil-4.61-cp37-none-any.whl size=3709861 sha256=a3bda14e54da14b39d9852213a363474a1e8dae0c951c1076a25fab14bf90b4b
  Stored in directory: /root/.cache/pip/wheels/21/c8/4c/b30f998197ac91

In [ ]:
import math
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import learn2learn as l2l

In [ ]:
class dataset(torch.utils.data.Dataset):    
    def __init__(self , train=True):
        self.data= pd.read_csv("/content/pendigits_csv.csv").to_numpy()
        if train :
          self.data = self.data[self.data[:,-1]<6]
        else :
          self.data = self.data[self.data[:,-1]>=6]


    def __getitem__(self, index):
        x, y = torch.tensor((self.data[index,:-1] - np.min(self.data[index,:-1] , axis=0)) / ((np.max(self.data[index,:-1], axis=0)) - np.min(self.data[index,:-1], axis=0)) ), torch.tensor(self.data[index,-1])

        return x,y

    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = dataset(train=True)
valid_dataset = dataset(train=False)

In [ ]:
from learn2learn.data.transforms import NWays, KShots, LoadData, RemapLabels, ConsecutiveLabels

train_dataset = l2l.data.MetaDataset(train_dataset)
valid_dataset = l2l.data.MetaDataset(valid_dataset)

train_transforms = [
        NWays(train_dataset, 4),
        KShots(train_dataset, 5*2),
        LoadData(train_dataset),
        RemapLabels(train_dataset),
        ConsecutiveLabels(train_dataset)
    ]

valid_transforms = [
        NWays(valid_dataset, 4),
        KShots(valid_dataset, 5*2),
        LoadData(valid_dataset),
        RemapLabels(valid_dataset),
        ConsecutiveLabels(valid_dataset)
    ]


train_tasks = l2l.data.TaskDataset(train_dataset,
                                      task_transforms=train_transforms,
                                      num_tasks=1000)

valid_tasks = l2l.data.TaskDataset(valid_dataset,
                                      task_transforms=valid_transforms,
                                      num_tasks=100)


In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

class Discriminator(nn.Module):
    
    def __init__(self):
        super(Discriminator,self).__init__()  
        self.fc1 = nn.Linear(16, 24)
        self.fc2 = nn.Linear(24,32)
        self.fc3 = nn.Linear(32,4+1)
        self.relu = nn.ReLU()
        self.activation = nn.Softmax(dim=1)

    def forward(self,x):


        x = self.fc3(  self.relu(self.fc2(self.relu(self.fc1(x))))  )
        
        D_output = self.activation(  x[:,:-1]  )

        last_logit = self.activation(x[:,-1].reshape(1,-1))


        return D_output , last_logit

In [ ]:
class Generator(nn.Module):
    
    def __init__(self , z):
        super(Generator,self).__init__()  
        self.z = z
        self.fc1 = nn.Linear( z + 16 , 24)
        self.fc2 = nn.Linear(24,18)
        self.fc3 = nn.Linear(18,16)
        self.relu = nn.ReLU()
        self.activation = nn.Sigmoid()

    def forward(self , encoding):

        x = torch.randn(self.z).to('cuda')

        inp= torch.cat((encoding,x)).type(torch.cuda.FloatTensor)
 
        G_output = self.activation(  self.fc3(  self.relu(self.fc2(self.relu(self.fc1(inp))))  )   )

        return G_output

In [ ]:
import random


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, generator, optimizer2 , train_ratio ,loss, loss1 ,adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]


    encoding = adaptation_data.mean(axis = 0)


    # Adapt the model
    for step in range(adaptation_steps):

        ###
        """
        encoding

        tbn 
        encoding size = ( shots*ways , 58)
        """
        ###

        ### Generator
        for i in range(train_ratio):
          
          fake_data = generator(encoding)
        
          _ , last_logit = learner(fake_data.reshape(-1,16))

          G_label = torch.zeros((1)).to(device)

          G_loss = loss1(last_logit.reshape(1) , G_label )
          
          optimizer2.zero_grad()

          G_loss.backward()
          optimizer2.step()
        
        optimizer2.zero_grad()

        ### Disc

        disc_logit , last_logit = learner(adaptation_data.type(torch.cuda.FloatTensor))
 
        task_train_error = loss(disc_logit.reshape(ways*shots , ways) , evaluation_labels.reshape(ways*shots) )

        real_loss = loss1(last_logit.reshape(ways*shots) , torch.zeros((ways*shots)).to(device)  )

        fake_data = generator(encoding)
        _ , last_logit = learner(fake_data.reshape(-1,16))
        
        G_label = torch.ones((1)).to(device)

        fake_loss = loss1(last_logit.reshape(1) , G_label )

        train_error = task_train_error + real_loss + fake_loss

        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions , _ = learner(evaluation_data.type(torch.cuda.FloatTensor))
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def main(
        ways=4,
        shots=5,
        meta_lr=0.01,
        fast_lr=0.5,
        gen_lr=0.1 ,
        meta_batch_size=32,
        adaptation_steps=1,
        train_ratio = 1, 
        num_iterations=60000,
        cuda=True,
        seed=42,
):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda:
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Create model
    model = Discriminator()
    
    generator = Generator(32)

    model.to(device)
    generator.to(device)

    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.RMSprop(maml.parameters(), meta_lr )

    optimizer2 = optim.RMSprop(generator.parameters(), gen_lr )
    
    loss = nn.CrossEntropyLoss(reduction='mean')

    loss1 = nn.BCELoss(reduction='mean')


    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = train_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               generator,
                                                               optimizer2,
                                                               train_ratio,
                                                               loss,
                                                               loss1,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = valid_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               generator,
                                                               optimizer2,
                                                               train_ratio,
                                                               loss,
                                                               loss1,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
        print('\n')
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)
        print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
        print('Meta Valid Error', meta_valid_error / meta_batch_size)
        print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    # meta_test_error = 0.0
    # meta_test_accuracy = 0.0
    # for task in range(meta_batch_size):
    #     # Compute meta-testing loss
    #     learner = maml.clone()
    #     batch = tasksets.test.sample()
    #     evaluation_error, evaluation_accuracy = fast_adapt(batch,
    #                                                        learner,
    #                                                        loss,
    #                                                        adaptation_steps,
    #                                                        shots,
    #                                                        ways,
    #                                                        device)
    #     meta_test_error += evaluation_error.item()
    #     meta_test_accuracy += evaluation_accuracy.item()
    # print('Meta Test Error', meta_test_error / meta_batch_size)
    # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)


if __name__ == '__main__':
    main()

Streaming output truncated to the last 5000 lines.
Meta Valid Error 1.1496146861463785
Meta Valid Accuracy 0.5953125115483999


Iteration 9153
Meta Train Error 0.7945529166609049
Meta Train Accuracy 0.9484375063329935
Meta Valid Error 1.1459223926067352
Meta Valid Accuracy 0.6046875156462193


Iteration 9154
Meta Train Error 0.7826703991740942
Meta Train Accuracy 0.9609375055879354
Meta Valid Error 1.1247226260602474
Meta Valid Accuracy 0.631250006146729


Iteration 9155
Meta Train Error 0.7776323668658733
Meta Train Accuracy 0.9671875033527613
Meta Valid Error 1.1346943713724613
Meta Valid Accuracy 0.6234375117346644


Iteration 9156
Meta Train Error 0.7991546057164669
Meta Train Accuracy 0.9453125055879354
Meta Valid Error 1.1377393435686827
Meta Valid Accuracy 0.6203125109896064


Iteration 9157
Meta Train Error 0.7881682738661766
Meta Train Accuracy 0.9531250055879354
Meta Valid Error 1.1476604156196117
Meta Valid Accuracy 0.6109375124797225


Iteration 9158
Meta Train Error 0.8061

KeyboardInterrupt: ignored

In [ ]:
import random


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data.type(torch.cuda.FloatTensor)).reshape(ways*shots,ways), adaptation_labels.reshape(ways*shots))
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data.type(torch.cuda.FloatTensor))
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def main(
        ways=4,
        shots=5,
        meta_lr=0.01,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=60000,
        cuda=True,
        seed=42,
):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda:
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Create model
    model = Discriminator()
    model.to(device)
    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.RMSprop(maml.parameters(), meta_lr )
    loss = nn.CrossEntropyLoss(reduction='mean')

    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = train_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = valid_tasks.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
        print('\n')
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)
        print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
        print('Meta Valid Error', meta_valid_error / meta_batch_size)
        print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    # meta_test_error = 0.0
    # meta_test_accuracy = 0.0
    # for task in range(meta_batch_size):
    #     # Compute meta-testing loss
    #     learner = maml.clone()
    #     batch = tasksets.test.sample()
    #     evaluation_error, evaluation_accuracy = fast_adapt(batch,
    #                                                        learner,
    #                                                        loss,
    #                                                        adaptation_steps,
    #                                                        shots,
    #                                                        ways,
    #                                                        device)
    #     meta_test_error += evaluation_error.item()
    #     meta_test_accuracy += evaluation_accuracy.item()
    # print('Meta Test Error', meta_test_error / meta_batch_size)
    # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)


if __name__ == '__main__':
    main()

Streaming output truncated to the last 5000 lines.
Meta Valid Error 1.2319830171763897
Meta Valid Accuracy 0.4984375131316483


Iteration 9287
Meta Train Error 0.815538389608264
Meta Train Accuracy 0.9265625011175871
Meta Valid Error 1.2651310376822948
Meta Valid Accuracy 0.46875000838190317


Iteration 9288
Meta Train Error 0.8350802697241306
Meta Train Accuracy 0.9078125078231096
Meta Valid Error 1.2310781106352806
Meta Valid Accuracy 0.5046875104308128


Iteration 9289
Meta Train Error 0.8179877642542124
Meta Train Accuracy 0.9234375059604645
Meta Valid Error 1.2558143585920334
Meta Valid Accuracy 0.48593750689178705


Iteration 9290
Meta Train Error 0.8266766089946032
Meta Train Accuracy 0.9171875100582838
Meta Valid Error 1.2427715361118317
Meta Valid Accuracy 0.47968750819563866


Iteration 9291
Meta Train Error 0.8600579239428043
Meta Train Accuracy 0.8828125074505806
Meta Valid Error 1.241952270269394
Meta Valid Accuracy 0.5078125093132257


Iteration 9292
Meta Train Error 0.81

KeyboardInterrupt: ignored

In [ ]:
# model2 = model.Generator().to('cuda')
# model1 = model.Discriminator().to('cuda')
# device='cuda'

# #optimizer 

# lr_init = 0.01
# # optimizer1 = torch.optim.Adam(model1.parameters(), lr = lr)
# # optimizer2 = torch.optim.Adam(model2.parameters(), lr = lr)

# import matplotlib.pyplot as plt
# for j in range(500):
#      for i , (image , label) in enumerate(train_loader):
         
#          lr = lr_init*(0.1)**(j//100)
#          optimizer1 = torch.optim.RMSprop(model1.parameters(), lr = lr)
#          optimizer2 = torch.optim.RMSprop(model2.parameters(), lr = lr)

#          optimizer1.zero_grad()
#          image.to('cuda')
         
#          Dloss = model1.loss(image , model2)
#          Dloss.backward()
#          optimizer1.step()
         
#          for k in range(3):
#            optimizer2.zero_grad()
     
#            Gloss , _  = model2.loss(image , model1)
#            Gloss.backward()
#            optimizer2.step()
           
#            if i%10 == 0:
#              print(j,Dloss)
#              z = (torch.randn(1, 100 , 1 , 1 ).to('cpu'))
#              output = model2(z)
#             #  plt.imshow(output.cpu().detach().numpy().reshape(4,1)  , cmap='Greys_r')
#              plt.show()   

0 tensor(1.5960, device='cuda:0', grad_fn=<AddBackward0>)
0 tensor(1.5960, device='cuda:0', grad_fn=<AddBackward0>)
0 tensor(1.5960, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(1.2055, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(1.2055, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(1.2055, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(1.1455, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(1.1455, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(1.1455, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(1.0819, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(1.0819, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(1.0819, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(1.0471, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(1.0471, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(1.0471, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(1.0251, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(1.0251, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(1.025